# Student Attendance, Academics and Discipline

In this Jupyter notebook, attendance data for students will be compiled that reflect the number of periods they were tardy as well as the number of periods they had an unexcused absence as well as the number of total absent periods. A total of discipline data logs is also calculated as well as the number of suspensions that they had. Finally, the academic data for students will be compiled by looking at their total academic GPA as well as the number of Ds and Fs that they are getting in classes currently.

In [ ]:
import numpy as np
import pandas as pd

The Aeries SIS queries for the data that is needed is included as well as uploading them into the sheet.

In [ ]:
#LIST STU SC ID STU.NM GR AHD AHD.YR AHD.AL AHD.P0 AHD.P1 AHD.P2 AHD.P3 AHD.P4 AHD.P5 AHD.P6 AHD.P7 AHD.P8 AHD.P9 AHD.CD IF AHD.CD = C OR AHD.CD = T OR AHD.CD = Z

tardies = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Tardies.xlsx")

#LIST STU SC ID STU.NM GR AHD AHD.YR AHD.AL AHD.P0 AHD.P1 AHD.P2 AHD.P3 AHD.P4 AHD.P5 AHD.P6 AHD.P7 AHD.P8 AHD.P9 AHD.CD

absences = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Absences.xlsx")

# LIST STU ID DIS DIS.CD? DIS.DT 
discipline = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Discipline.xlsx")

# LIST STU ADS DSP ADS.SC STU.ID ADS.DT ADS.CD DSP.DS DSP.DD
suspension = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Suspension.xlsx")

# LIST STU SC ID STU.NM GR TP

gpa = pd.read_excel(r"C:\Users\derek.castleman\Desktop\GPA.xlsx")

# LIST STU ID FEE FEE.AM FEE.DT FEE.PD FEE.CD?

fees = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Fees.xlsx")

# LIST STU GBU GBK GBR STU.SC STU.ID STU.NM STU.GR GBK.PD GBK.NM GBR.CD GBU.CMK GBU.TG GBU.CSC

gradebook = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Gradebook.xlsx")

## Attendance

Attendance will be looked at in three different ways. First, the number of periods that a student was tardy to class will be summed up. I will follow this by calculating the number of periods that each student missed due to an unexcused absence. It will conclude by calculating the total periods that a student missed whether or not they were excused or unexcused.

### Tardies

In [ ]:
tardies

In [ ]:
x = input('Input year of interest (2022-2023, 2023-2024, etc.)        ') # Inputting current year of interest
x

In [ ]:
year_tardies = tardies[tardies['Year'] == x] #Selecting the current year out of the dataset
year_tardies

T and Z oftentime appear as an all day code and a period code which would cause a doubling of the numbers for the students. so these have to be dealt with separately than normal class codes.

In [ ]:
fix_tardies = year_tardies[(year_tardies['Code'] == 'T') | (year_tardies['Code'] == 'Z')] # Selecting T and Z
fix_tardies

In [ ]:
# Summing all of the columns
fix_tardies = fix_tardies.assign(sum_column=fix_tardies['Per 0'] + fix_tardies['Per 1'] + fix_tardies['Per 2']
                                 + fix_tardies['Per 3'] + fix_tardies['Per 4'] + fix_tardies['Per 5']
                                 + fix_tardies['Per 6'] + fix_tardies['Per 7'] + fix_tardies['Per 8'] + fix_tardies['Per 9'])
fix_tardies

In [ ]:
# Assigning sum column to all day
fix_tardies['All'] = np.where(fix_tardies['All'] ==0, fix_tardies['sum_column'], fix_tardies['All'])
fix_tardies

In [ ]:
# Turning the rest of the columns to zero
columns_to_zero = ['Per 0', 'Per 1', 'Per 2', 'Per 3', 'Per 4', 'Per 5', 'Per 6', 'Per 7', 'Per 8', 'Per 9']
fix_tardies[columns_to_zero] = 0
fix_tardies

In [ ]:
# Dropping sum column
fix_tardies = fix_tardies.drop(['sum_column'], axis=1)
fix_tardies

In [ ]:
# Looking at class tardies
class_tardies = year_tardies[year_tardies['Code'] == 'C']
class_tardies

In [ ]:
# Summing the class tardies
class_tardies = class_tardies.assign(sum_column=class_tardies['Per 0'] + class_tardies['Per 1'] + class_tardies['Per 2']
                                 + class_tardies['Per 3'] + class_tardies['Per 4'] + class_tardies['Per 5']
                                 + class_tardies['Per 6'] + class_tardies['Per 7'] + class_tardies['Per 8']
                                     + class_tardies['Per 9'])
class_tardies

In [ ]:
# Assigning sum column to all day
class_tardies['All'] = np.where(class_tardies['All'] >=0, class_tardies['sum_column'], class_tardies['All'])
class_tardies

In [ ]:
# Dropping sum column
class_tardies = class_tardies.drop(['sum_column'], axis=1)
class_tardies

In [ ]:
# Putting the two tardy data frames together
merged = pd.concat([class_tardies, fix_tardies])
merged

In [ ]:
# Grouping by student ID and summing all days to get total tardies
merged = merged.groupby(by=['Student ID'])['All'].sum().reset_index()
merged

In [ ]:
tardies = merged.rename(columns={'All': 'Period Tardies'}) # Renaming tardies column
tardies

### Absences

In [ ]:
absences

In [ ]:
absences = absences[absences['Year'] == x] # Selecting the year of interest
absences

In [ ]:
# Selecting the unexcused absent codes
unexcused = absences[(absences['Code'] == 'A') | (absences['Code'] == 'Q') | 
                     (absences['Code'] == 'S') | (absences['Code'] == 'U')]
unexcused

In [ ]:
# Setting All Day to zero to not overcount
columns_to_zero = ['All']
unexcused[columns_to_zero] = 0
unexcused

In [ ]:
# Summing all of the periods
unexcused = unexcused.assign(unexcused_periods=unexcused['Per 0'] + unexcused['Per 1'] + unexcused['Per 2']
                                 + unexcused['Per 3'] + unexcused['Per 4'] + unexcused['Per 5']
                                 + unexcused['Per 6'] + unexcused['Per 7'] + unexcused['Per 8']
                                     + unexcused['Per 9'])
unexcused

In [ ]:
# Grouping by student ID and summing the totals category
unexcused = unexcused.groupby(by=['Student ID'])['unexcused_periods'].sum().reset_index()
unexcused

In [ ]:
# Renaming the column
unexcused = unexcused.rename(columns={'unexcused_periods': 'Unexcused Absent Periods'})
unexcused

In [ ]:
# Selecting all of the absent codes
totalabsent = absences[(absences['Code'] == 'A') | (absences['Code'] == 'Q') | 
                     (absences['Code'] == 'S') | (absences['Code'] == 'U') | (absences['Code'] == 'K') |
                      (absences['Code'] == 'I') | (absences['Code'] == 'M') | (absences['Code'] == 'X')]
totalabsent

In [ ]:
# Summing the columns
columns_to_zero = ['All']
totalabsent[columns_to_zero] = 0
totalabsent

In [ ]:
# Summing all of the periods
totalabsent = totalabsent.assign(absent_periods=totalabsent['Per 0'] + totalabsent['Per 1'] + totalabsent['Per 2']
                                 + totalabsent['Per 3'] + totalabsent['Per 4'] + totalabsent['Per 5']
                                 + totalabsent['Per 6'] + totalabsent['Per 7'] + totalabsent['Per 8']
                                     + totalabsent['Per 9'])
totalabsent

In [ ]:
# Grouping by student ID and summing the totals
totalabsent = totalabsent.groupby(by=['Student ID'])['absent_periods'].sum().reset_index()
totalabsent

In [ ]:
totalabsent = totalabsent.rename(columns={'absent_periods': 'Total Absent Periods'})
totalabsent

In [ ]:
# Merging period tardies with unexcused absences
attendance = pd.merge(tardies, unexcused, how='outer', on='Student ID')
attendance

In [ ]:
# Merging with total absent
attendance = pd.merge(attendance, totalabsent, how='outer', on='Student ID')
attendance

In [ ]:
# Filling in na columns with zeros
attendance = attendance.fillna(0)
attendance

## Discipline

A date will be input that will be selected for when discipline should be looked at beyond. A number of one will be assigned to each discipline log and then it will be summed for each of the students. The same thing will be done with the suspensions for a student.

In [ ]:
discipline

In [ ]:
discipline['Date']= pd.to_datetime(discipline['Date']) # Changing to proper date format
discipline

In [ ]:
y = input('Input first day of school year mm/dd/yyyy        ') # Selecting start date for discipline
y

In [ ]:
y = pd.to_datetime(y) # Changing start date into proper format
y

In [ ]:
discipline = discipline[discipline['Date'] > y] # Selecting discipline beyond start date
discipline

In [ ]:
discipline['Discipline Logs'] = 1 # Giving a value of one for each discipline
discipline

In [ ]:
# Summing up the discipline for each student
discipline = discipline.groupby(by=['Student ID'])['Discipline Logs'].sum().reset_index()
discipline

In [ ]:
suspension

In [ ]:
suspension = suspension[suspension['Disposition'] == 'SUS-13'] # Selecting suspensions
suspension

In [ ]:
# Changing date into proper format
suspension['Disposition date']= pd.to_datetime(suspension['Disposition date'])
suspension

In [ ]:
# Selecting suspensions beyond the start date
suspension = suspension[suspension['Disposition date'] > y]
suspension

In [ ]:
suspension['Suspensions'] = 1 # Giving a value of 1 for each suspension
suspension

In [ ]:
# Summing up the suspensions for each student
suspension = suspension.groupby(by=['Student ID'])['Suspensions'].sum().reset_index()
suspension

In [ ]:
# Merging the two behavior columns together
behavior = pd.merge(discipline, suspension, how='outer', on='Student ID')
behavior

In [ ]:
behavior = behavior.fillna(0) # Filling in the missing values with zero
behavior

## Academics

The total academic GPA for students will be added to the attendance data along with any outstanding fees that they might have. The outstanding fees are calculated by the total they have been charged with the total that has been paid minused from it. The grades for the current term will be selected and any grade under a C- will be chosen with a sum for each student for how many Ds and Fs they have given.

In [ ]:
gpa

In [ ]:
# Adds the total GPA of each student with their attendance data
merge_one = pd.merge(gpa, attendance, how='left', on='Student ID')
merge_one

In [ ]:
final = pd.merge(merge_one, behavior, how='left', on='Student ID')
final

In [ ]:
final = final.fillna(0)
final

In [ ]:
fees

In [ ]:
# Debts are calculated by subtracting amount paid from amount charged
fees['Fees/Debts'] = fees['Amount Chgd'] - fees['Amount Pd']
fees

In [ ]:
fees = fees[['Student ID', 'Fees/Debts']]
fees

In [ ]:
gradebook

In [ ]:
z = input('Input term of interest (1, 2 etc.)        ') # Current term for grades is unput
z

In [ ]:
gradebook = gradebook[gradebook['Term'] == z] # Current term is selected from gradebook
gradebook

In [ ]:
# Failing grades are chosen
failing = gradebook[(gradebook['OverallMark'] == 'D+') | (gradebook['OverallMark'] == 'D') | 
                    (gradebook['OverallMark'] == 'D-') | (gradebook['OverallMark'] == 'F')]
failing

In [ ]:
# A value of 1 is given to each course that has been filtered
failing['D/F'] = 1
failing

In [ ]:
# Total number is summed up
failing = failing.groupby(by=['Student ID'])['D/F'].sum().reset_index()
failing

In [ ]:
final

In [ ]:
# Failing grades are added to all other merges
final = pd.merge(final, failing, on='Student ID', how='left')
final

In [ ]:
# Fees are added to the end
final = pd.merge(final, fees, on='Student ID', how='left')
final

In [ ]:
# Any remaining nan are filled in with 0
final = final.fillna(0)
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Attendance and Discipline", filename = "Attendance and Discipline"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)